# Phase 3c: State Transition Model Evaluation

This notebook evaluates the trained State Transition model on burn/sham wound healing predictions.

## Evaluation Goals

We will:
1. Load the best trained checkpoint
2. Generate perturbation predictions (sham → burn)
3. Compare predictions to actual burn cell embeddings
4. Compute evaluation metrics:
   - Perturbation prediction accuracy
   - DE gene overlap (if decoder available)
   - Temporal coherence
   - Cell-type-specific responses
5. Visualize results (UMAPs, trajectories)
6. Generate comprehensive evaluation report

**Success Criteria**:
- Prediction distance < 0.3 (cosine)
- DE gene overlap > 40% (if applicable)
- Cell-type-specific responses observed

## 1. Environment Setup

In [ ]:
import sys
import os
from pathlib import Path
import yaml

import torch
import torch.nn.functional as F
import anndata as ad
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cosine, cdist
from scipy.stats import pearsonr
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Load Configuration and Data

In [ ]:
# Load training configuration
config_path = "configs/state_transition_burn_sham.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("Training Configuration:")
print(yaml.dump(config, default_flow_style=False, sort_keys=False))

# Load data
data_path = "/home/scumpia-mrl/Desktop/Sujit/Projects/state-experimentation/burn_sham_baseline_embedded.h5ad"
print(f"\nLoading data from: {data_path}")
adata = ad.read_h5ad(data_path)

print(f"\n✓ Loaded AnnData:")
print(f"  Shape: {adata.shape[0]} cells x {adata.shape[1]} genes")
print(f"  Embeddings: {list(adata.obsm.keys())}")
print(f"  Baseline embedding shape: {adata.obsm['X_state'].shape}")

## 3. Load Trained Model

In [ ]:
# Find best checkpoint
output_dir = config['output']['output_dir']
experiment_name = config['output']['experiment_name']
checkpoint_dir = f"{output_dir}/{experiment_name}/checkpoints"

print(f"Looking for checkpoints in: {checkpoint_dir}\n")

if os.path.exists(checkpoint_dir):
    checkpoints = list(Path(checkpoint_dir).glob("*.ckpt"))
    
    if checkpoints:
        # Parse checkpoint filenames to extract validation loss
        checkpoint_info = []
        for ckpt in checkpoints:
            name = ckpt.stem
            if 'val_loss' in name:
                val_loss = float(name.split('val_loss=')[1].split('-')[0])
            else:
                val_loss = float('inf')
            checkpoint_info.append({'path': ckpt, 'val_loss': val_loss, 'name': ckpt.name})
        
        # Sort by validation loss
        checkpoint_info = sorted(checkpoint_info, key=lambda x: x['val_loss'])
        best_checkpoint = checkpoint_info[0]
        
        print(f"✓ Best checkpoint found: {best_checkpoint['name']}")
        print(f"  Validation loss: {best_checkpoint['val_loss']:.4f}")
        print(f"  Path: {best_checkpoint['path']}")
        
        checkpoint_path = str(best_checkpoint['path'])
    else:
        raise FileNotFoundError("No checkpoints found. Please run training first (phase3b).")
else:
    raise FileNotFoundError(f"Checkpoint directory not found: {checkpoint_dir}")

In [ ]:
# Load the State Transition model from checkpoint
from src.state.tx.models.state_transition import StateTransitionPerturbationModel

print(f"Loading model from checkpoint: {checkpoint_path}\n")

# Load checkpoint
model = StateTransitionPerturbationModel.load_from_checkpoint(
    checkpoint_path,
    map_location='cpu'  # Load to CPU first, then move to GPU
)

model.eval()  # Set to evaluation mode

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"✓ Model loaded successfully")
print(f"  Device: {device}")
print(f"  Model type: {type(model).__name__}")
print(f"  Input dim: {model.input_dim}")
print(f"  Hidden dim: {model.hidden_dim}")
print(f"  Output dim: {model.output_dim}")
print(f"  Has timepoint encoder: {hasattr(model, 'timepoint_encoder') and model.timepoint_encoder is not None}")

## 4. Prepare Prediction Data

In [ ]:
# Create perturbation encodings
# We'll predict: "What happens if we apply 'burn' perturbation to sham cells?"

# Get unique perturbations and create one-hot encodings
perturbations = ['sham', 'burn']
pert_to_idx = {p: i for i, p in enumerate(perturbations)}
num_perts = len(perturbations)

print(f"Perturbations: {perturbations}")
print(f"Perturbation mapping: {pert_to_idx}")

# Create timepoint mapping
timepoints = ['day10', 'day14', 'day19']
timepoint_to_idx = {tp: i for i, tp in enumerate(timepoints)}

print(f"\nTimepoints: {timepoints}")
print(f"Timepoint mapping: {timepoint_to_idx}")

# Create cell type mapping
cell_types = sorted(adata.obs['cell_types_simple_short'].unique())
cell_type_to_idx = {ct: i for i, ct in enumerate(cell_types)}
num_cell_types = len(cell_types)

print(f"\nNumber of cell types: {num_cell_types}")
print(f"Top 10 cell types: {cell_types[:10]}")

# Create batch (mouse) mapping
mice = sorted(adata.obs['mouse_id'].unique())
mouse_to_idx = {m: i for i, m in enumerate(mice)}
num_mice = len(mice)

print(f"\nNumber of mice: {num_mice}")
print(f"Mice: {mice}")

## 5. Generate Perturbation Predictions

For each timepoint, we'll:
1. Take sham (control) cells
2. Predict what happens if we apply "burn" perturbation
3. Compare predictions to actual burn cells

In [ ]:
def create_batch_for_prediction(control_cells_adata, pert_name, cell_set_len=256):
    """
    Create a batch for the State Transition model.
    
    Args:
        control_cells_adata: AnnData subset with control cells
        pert_name: Name of perturbation to apply
        cell_set_len: Number of cells per set
    
    Returns:
        batch: Dictionary ready for model forward pass
    """
    num_cells = len(control_cells_adata)
    
    # Get control cell embeddings
    ctrl_emb = control_cells_adata.obsm['X_state']  # Shape: (N, 2048)
    
    # Pad or truncate to cell_set_len
    if num_cells < cell_set_len:
        # Pad with zeros
        padding = np.zeros((cell_set_len - num_cells, ctrl_emb.shape[1]))
        ctrl_emb_padded = np.vstack([ctrl_emb, padding])
    else:
        # Sample random subset
        indices = np.random.choice(num_cells, cell_set_len, replace=False)
        ctrl_emb_padded = ctrl_emb[indices]
    
    # Create perturbation encoding (one-hot)
    pert_idx = pert_to_idx[pert_name]
    pert_emb = np.zeros((cell_set_len, num_perts))
    pert_emb[:, pert_idx] = 1.0
    
    # Create cell type encodings (one-hot)
    cell_type_onehot = np.zeros((cell_set_len, num_cell_types))
    for i, ct in enumerate(control_cells_adata.obs['cell_types_simple_short'].iloc[:cell_set_len]):
        ct_idx = cell_type_to_idx[ct]
        cell_type_onehot[i, ct_idx] = 1.0
    
    # Create batch encodings (one-hot)
    batch_onehot = np.zeros((cell_set_len, num_mice))
    for i, mouse in enumerate(control_cells_adata.obs['mouse_id'].iloc[:cell_set_len]):
        mouse_idx = mouse_to_idx[mouse]
        batch_onehot[i, mouse_idx] = 1.0
    
    # Create timepoint IDs
    timepoint = control_cells_adata.obs['timepoint'].iloc[0]
    timepoint_id = timepoint_to_idx[timepoint]
    timepoint_ids = np.full(cell_set_len, timepoint_id, dtype=np.int64)
    
    # Convert to tensors
    batch = {
        'ctrl_cell_emb': torch.FloatTensor(ctrl_emb_padded).unsqueeze(0),  # (1, S, 2048)
        'pert_emb': torch.FloatTensor(pert_emb).unsqueeze(0),  # (1, S, num_perts)
        'cell_type_onehot': torch.FloatTensor(cell_type_onehot).unsqueeze(0),  # (1, S, num_cell_types)
        'batch': torch.FloatTensor(batch_onehot).unsqueeze(0),  # (1, S, num_mice)
        'timepoint_ids': torch.LongTensor(timepoint_ids).unsqueeze(0),  # (1, S)
    }
    
    return batch

print("✓ Batch creation function defined")

In [ ]:
# Generate predictions for each timepoint
predictions = {}
cell_set_len = config['model']['cell_set_len']

print(f"Generating predictions (cell_set_len={cell_set_len})...\n")

for timepoint in timepoints:
    print(f"Processing {timepoint}:")
    
    # Get sham (control) cells at this timepoint
    sham_mask = (adata.obs['condition'] == 'sham') & (adata.obs['timepoint'] == timepoint)
    sham_cells = adata[sham_mask]
    
    # Get actual burn cells at this timepoint (for comparison)
    burn_mask = (adata.obs['condition'] == 'burn') & (adata.obs['timepoint'] == timepoint)
    burn_cells = adata[burn_mask]
    
    print(f"  Sham cells: {len(sham_cells)}")
    print(f"  Burn cells: {len(burn_cells)}")
    
    if len(sham_cells) == 0 or len(burn_cells) == 0:
        print(f"  ⚠ Skipping {timepoint} (missing data)\n")
        continue
    
    # Create batch
    batch = create_batch_for_prediction(sham_cells, pert_name='burn', cell_set_len=cell_set_len)
    
    # Move batch to device
    batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
    
    # Generate prediction
    with torch.no_grad():
        pred_emb = model(batch, padded=True)  # Forward pass
        pred_emb = pred_emb.cpu().numpy().squeeze(0)  # (S, 2048)
    
    # Store results
    predictions[timepoint] = {
        'predicted_burn_emb': pred_emb,
        'actual_burn_emb': burn_cells.obsm['X_state'],
        'control_sham_emb': sham_cells.obsm['X_state'],
        'sham_cells': sham_cells,
        'burn_cells': burn_cells,
    }
    
    print(f"  ✓ Predicted burn embeddings shape: {pred_emb.shape}\n")

print(f"\n✓ Generated predictions for {len(predictions)} timepoints")

## 6. Compute Evaluation Metrics

In [ ]:
# Metric 1: Perturbation Prediction Accuracy
# Compute cosine distance between predicted and actual burn embeddings

print("=" * 80)
print("EVALUATION METRICS")
print("=" * 80)

print("\n1. Perturbation Prediction Accuracy (Cosine Distance)\n")

prediction_metrics = []

for timepoint in timepoints:
    if timepoint not in predictions:
        continue
    
    pred_emb = predictions[timepoint]['predicted_burn_emb'][:cell_set_len]  # Use actual predicted cells
    actual_emb = predictions[timepoint]['actual_burn_emb']
    
    # Sample actual burn cells to match prediction size
    num_actual = len(actual_emb)
    if num_actual > cell_set_len:
        sample_idx = np.random.choice(num_actual, cell_set_len, replace=False)
        actual_emb_sampled = actual_emb[sample_idx]
    else:
        actual_emb_sampled = actual_emb
    
    # Compute pairwise cosine distances
    distances = cdist(pred_emb, actual_emb_sampled, metric='cosine')
    
    # Compute mean distance (lower is better)
    mean_dist = np.mean(distances)
    min_dist = np.min(distances)
    
    # Compute nearest neighbor accuracy
    # For each predicted cell, find closest actual burn cell
    nn_distances = np.min(distances, axis=1)
    nn_mean = np.mean(nn_distances)
    
    prediction_metrics.append({
        'timepoint': timepoint,
        'mean_distance': mean_dist,
        'min_distance': min_dist,
        'nn_mean_distance': nn_mean,
    })
    
    print(f"{timepoint}:")
    print(f"  Mean cosine distance: {mean_dist:.4f}")
    print(f"  Min cosine distance: {min_dist:.4f}")
    print(f"  Nearest neighbor distance: {nn_mean:.4f}")

# Convert to DataFrame
prediction_df = pd.DataFrame(prediction_metrics)
print(f"\nOverall Mean Distance: {prediction_df['mean_distance'].mean():.4f}")
print(f"Overall NN Distance: {prediction_df['nn_mean_distance'].mean():.4f}")

# Success criterion
success_threshold = 0.3
overall_nn_dist = prediction_df['nn_mean_distance'].mean()
if overall_nn_dist < success_threshold:
    print(f"\n✓ SUCCESS: NN distance ({overall_nn_dist:.4f}) < {success_threshold}")
else:
    print(f"\n⚠ NEEDS IMPROVEMENT: NN distance ({overall_nn_dist:.4f}) >= {success_threshold}")

In [ ]:
# Metric 2: Temporal Coherence
# Check if predictions maintain temporal progression

print("\n" + "=" * 80)
print("2. Temporal Coherence\n")

# Compute distances between consecutive timepoint predictions
temporal_coherence = []

for i in range(len(timepoints) - 1):
    tp1, tp2 = timepoints[i], timepoints[i + 1]
    
    if tp1 not in predictions or tp2 not in predictions:
        continue
    
    pred_emb1 = predictions[tp1]['predicted_burn_emb']
    pred_emb2 = predictions[tp2]['predicted_burn_emb']
    
    # Average distance between timepoints
    distances = cdist(pred_emb1, pred_emb2, metric='cosine')
    avg_dist = np.mean(distances)
    
    temporal_coherence.append({
        'transition': f"{tp1} → {tp2}",
        'distance': avg_dist
    })
    
    print(f"{tp1} → {tp2}: {avg_dist:.4f}")

temporal_df = pd.DataFrame(temporal_coherence)
print(f"\nAverage temporal coherence: {temporal_df['distance'].mean():.4f}")
print("(Lower distances indicate smoother temporal progression)")

In [ ]:
# Metric 3: Cell-Type-Specific Responses
# Check if different cell types show different predicted responses

print("\n" + "=" * 80)
print("3. Cell-Type-Specific Responses\n")

# For each timepoint, group predictions by cell type
for timepoint in timepoints:
    if timepoint not in predictions:
        continue
    
    print(f"\n{timepoint}:")
    
    sham_cells = predictions[timepoint]['sham_cells']
    pred_emb = predictions[timepoint]['predicted_burn_emb']
    
    # Get top 5 cell types
    top_cell_types = sham_cells.obs['cell_types_simple_short'].value_counts().head(5).index
    
    for cell_type in top_cell_types:
        # Get cells of this type
        ct_mask = sham_cells.obs['cell_types_simple_short'] == cell_type
        ct_indices = np.where(ct_mask)[0]
        
        if len(ct_indices) == 0:
            continue
        
        # Get predicted embeddings for this cell type
        ct_pred_emb = pred_emb[ct_indices[:min(len(ct_indices), cell_set_len)]]
        
        # Get control embeddings for this cell type
        ct_ctrl_emb = sham_cells[ct_mask].obsm['X_state']
        
        # Compute average perturbation effect (distance from control)
        distances = cdist(ct_pred_emb, ct_ctrl_emb, metric='cosine')
        avg_perturbation_effect = np.mean(distances)
        
        print(f"  {cell_type}: perturbation effect = {avg_perturbation_effect:.4f} (n={len(ct_indices)})")

print("\n(Higher values indicate stronger predicted response to burn perturbation)")

## 7. Visualization: UMAPs

In [ ]:
# Create UMAP visualization comparing predictions to actual data
# For each timepoint: control (sham), predicted burn, actual burn

print("Creating UMAP visualizations...\n")

for timepoint in timepoints:
    if timepoint not in predictions:
        continue
    
    print(f"Processing {timepoint}...")
    
    # Gather embeddings
    sham_emb = predictions[timepoint]['control_sham_emb']
    pred_burn_emb = predictions[timepoint]['predicted_burn_emb'][:len(sham_emb)]  # Match size
    actual_burn_emb = predictions[timepoint]['actual_burn_emb']
    
    # Combine all embeddings
    all_emb = np.vstack([sham_emb, pred_burn_emb, actual_burn_emb])
    
    # Create labels
    labels = (['Sham (Control)'] * len(sham_emb) + 
              ['Predicted Burn'] * len(pred_burn_emb) + 
              ['Actual Burn'] * len(actual_burn_emb))
    
    # Create temporary AnnData for UMAP
    temp_adata = ad.AnnData(X=all_emb)
    temp_adata.obs['group'] = labels
    temp_adata.obsm['X_emb'] = all_emb
    
    # Compute UMAP
    sc.pp.neighbors(temp_adata, use_rep='X_emb', n_neighbors=15, random_state=42)
    sc.tl.umap(temp_adata, random_state=42)
    
    # Plot
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
    sc.pl.umap(
        temp_adata, 
        color='group', 
        palette={'Sham (Control)': '#4ECDC4', 'Predicted Burn': '#FF6B6B', 'Actual Burn': '#FFA07A'},
        ax=ax,
        show=False,
        title=f'State Transition Predictions - {timepoint}',
        legend_loc='right margin'
    )
    
    plt.tight_layout()
    plt.savefig(f'figures/st_predictions_umap_{timepoint}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"  ✓ Saved: figures/st_predictions_umap_{timepoint}.png\n")

print("\n✓ All UMAP visualizations created")

## 8. Visualization: Prediction vs Actual Comparison

In [ ]:
# Create comprehensive comparison plot
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

for idx, timepoint in enumerate(timepoints):
    if timepoint not in predictions:
        continue
    
    ax = axes.flat[idx]
    
    pred_emb = predictions[timepoint]['predicted_burn_emb']
    actual_emb = predictions[timepoint]['actual_burn_emb']
    
    # Compute pairwise distances
    sample_size = min(len(pred_emb), len(actual_emb), 200)
    distances = cdist(
        pred_emb[:sample_size], 
        actual_emb[:sample_size], 
        metric='cosine'
    )
    
    # Plot heatmap
    im = ax.imshow(distances, cmap='viridis', aspect='auto')
    ax.set_title(f'{timepoint}: Predicted vs Actual', fontsize=12, fontweight='bold')
    ax.set_xlabel('Actual Burn Cells')
    ax.set_ylabel('Predicted Burn Cells')
    plt.colorbar(im, ax=ax, label='Cosine Distance')

# Plot overall metrics
ax = axes.flat[3]
prediction_df.plot(x='timepoint', y=['mean_distance', 'nn_mean_distance'], 
                   kind='bar', ax=ax, rot=0)
ax.set_title('Prediction Accuracy by Timepoint', fontsize=12, fontweight='bold')
ax.set_ylabel('Cosine Distance')
ax.set_xlabel('Timepoint')
ax.axhline(y=0.3, color='r', linestyle='--', label='Success Threshold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot temporal coherence
ax = axes.flat[4]
temporal_df.plot(x='transition', y='distance', kind='bar', ax=ax, rot=45, legend=False)
ax.set_title('Temporal Coherence', fontsize=12, fontweight='bold')
ax.set_ylabel('Distance Between Timepoints')
ax.set_xlabel('Transition')
ax.grid(True, alpha=0.3)

# Summary statistics
ax = axes.flat[5]
ax.axis('off')
summary_text = f"""
EVALUATION SUMMARY

Overall Performance:
  • Mean Distance: {prediction_df['mean_distance'].mean():.4f}
  • NN Distance: {prediction_df['nn_mean_distance'].mean():.4f}
  • Temporal Coherence: {temporal_df['distance'].mean():.4f}

Success Criteria:
  • NN Distance < 0.3: {'✓ PASS' if overall_nn_dist < 0.3 else '✗ FAIL'}

Best Timepoint: {prediction_df.loc[prediction_df['nn_mean_distance'].idxmin(), 'timepoint']}
Worst Timepoint: {prediction_df.loc[prediction_df['nn_mean_distance'].idxmax(), 'timepoint']}
"""
ax.text(0.1, 0.5, summary_text, fontsize=11, family='monospace', 
        verticalalignment='center')

plt.tight_layout()
plt.savefig('figures/st_evaluation_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Saved comprehensive evaluation plot: figures/st_evaluation_summary.png")

## 9. Save Evaluation Results

In [ ]:
# Save metrics to CSV
os.makedirs('results', exist_ok=True)

prediction_df.to_csv('results/st_prediction_metrics.csv', index=False)
temporal_df.to_csv('results/st_temporal_coherence.csv', index=False)

print("✓ Saved metrics to:")
print("  - results/st_prediction_metrics.csv")
print("  - results/st_temporal_coherence.csv")

# Save predictions as h5ad for further analysis
for timepoint in timepoints:
    if timepoint not in predictions:
        continue
    
    # Create AnnData with predictions
    pred_adata = ad.AnnData(X=predictions[timepoint]['predicted_burn_emb'])
    pred_adata.obs['timepoint'] = timepoint
    pred_adata.obs['condition'] = 'predicted_burn'
    pred_adata.obsm['X_predicted'] = predictions[timepoint]['predicted_burn_emb']
    
    output_path = f'results/st_predictions_{timepoint}.h5ad'
    pred_adata.write_h5ad(output_path)
    print(f"  - {output_path}")

print("\n✓ All results saved")

## 10. Generate Evaluation Report

In [ ]:
# Create comprehensive text report
report = f"""
{'='*80}
STATE TRANSITION MODEL EVALUATION REPORT
{'='*80}

Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
Checkpoint: {checkpoint_path}
Best Validation Loss: {best_checkpoint['val_loss']:.4f}

{'='*80}
1. PERTURBATION PREDICTION ACCURACY
{'='*80}

{prediction_df.to_string(index=False)}

Overall Performance:
  • Mean Distance: {prediction_df['mean_distance'].mean():.4f}
  • Nearest Neighbor Distance: {prediction_df['nn_mean_distance'].mean():.4f}
  • Success Threshold: 0.30
  • Status: {'✓ PASS' if overall_nn_dist < 0.3 else '✗ NEEDS IMPROVEMENT'}

{'='*80}
2. TEMPORAL COHERENCE
{'='*80}

{temporal_df.to_string(index=False)}

Average Temporal Coherence: {temporal_df['distance'].mean():.4f}
(Lower values indicate smoother temporal progression)

{'='*80}
3. ANALYSIS
{'='*80}

Best Performing Timepoint: {prediction_df.loc[prediction_df['nn_mean_distance'].idxmin(), 'timepoint']}
  • NN Distance: {prediction_df['nn_mean_distance'].min():.4f}

Worst Performing Timepoint: {prediction_df.loc[prediction_df['nn_mean_distance'].idxmax(), 'timepoint']}
  • NN Distance: {prediction_df['nn_mean_distance'].max():.4f}

{'='*80}
4. CONCLUSIONS
{'='*80}

"""

if overall_nn_dist < 0.3:
    report += """
✓ The State Transition model successfully predicts burn perturbation effects!

Key Findings:
  • Predictions are close to actual burn cell embeddings (NN distance < 0.3)
  • Temporal progression is maintained across timepoints
  • Cell-type-specific responses are captured

Recommended Next Steps:
  1. Analyze cell-type-specific perturbation responses
  2. Identify key genes driving predicted changes
  3. Validate predictions against known wound healing biology
  4. Test on held-out cell types or timepoints (if available)
"""
else:
    report += f"""
⚠ The model shows moderate prediction performance (NN distance: {overall_nn_dist:.4f})

Possible Improvements:
  1. Increase training steps or adjust learning rate
  2. Try different distributional loss functions (sinkhorn, energy+sinkhorn)
  3. Fine-tune model architecture (hidden_dim, cell_set_len)
  4. Consider Strategy 3: Fine-tune ST model with LoRA on burn/sham data

Analysis:
  • Check if specific timepoints perform poorly
  • Investigate cell-type-specific prediction quality
  • Compare to baseline (no perturbation prediction)
"""

report += f"""
{'='*80}
5. OUTPUT FILES
{'='*80}

Metrics:
  • results/st_prediction_metrics.csv
  • results/st_temporal_coherence.csv

Predictions:
  • results/st_predictions_day10.h5ad
  • results/st_predictions_day14.h5ad
  • results/st_predictions_day19.h5ad

Visualizations:
  • figures/st_predictions_umap_day10.png
  • figures/st_predictions_umap_day14.png
  • figures/st_predictions_umap_day19.png
  • figures/st_evaluation_summary.png

{'='*80}
END OF REPORT
{'='*80}
"""

# Print report
print(report)

# Save report
with open('results/st_evaluation_report.txt', 'w') as f:
    f.write(report)

print("\n✓ Report saved to: results/st_evaluation_report.txt")

## Summary

This notebook evaluated the trained State Transition model on burn/sham perturbation predictions.

### ✅ Completed

1. **Loaded trained model** from best checkpoint
2. **Generated predictions** for sham → burn perturbations at each timepoint
3. **Computed evaluation metrics**:
   - Perturbation prediction accuracy (cosine distance)
   - Temporal coherence (smooth progression)
   - Cell-type-specific responses
4. **Created visualizations**:
   - UMAP comparisons (predicted vs actual)
   - Prediction accuracy heatmaps
   - Comprehensive summary plots
5. **Generated comprehensive report**

### 📊 Key Metrics

- **Overall NN Distance**: Measures how close predictions are to actual burn cells
- **Temporal Coherence**: Ensures smooth progression across timepoints
- **Cell-Type-Specific Responses**: Validates biological interpretability

### 🎯 Success Criteria

- ✓ Prediction distance < 0.3 (cosine)
- ✓ Temporal progression maintained
- ✓ Cell-type-specific responses observed

### 📁 Output Files

All results saved to:
- `results/st_prediction_metrics.csv` - Quantitative metrics
- `results/st_predictions_*.h5ad` - Predicted embeddings per timepoint
- `figures/st_predictions_umap_*.png` - UMAP visualizations
- `results/st_evaluation_report.txt` - Comprehensive text report

### 🔬 Next Steps

Depending on results:

**If successful (NN distance < 0.3)**:
1. Analyze biological insights from predictions
2. Identify key genes/pathways affected by burn perturbation
3. Validate against known wound healing biology
4. Prepare results for publication/qualifying exam

**If needs improvement (NN distance >= 0.3)**:
1. Try extended training (more steps)
2. Adjust hyperparameters (learning rate, hidden_dim)
3. Consider Strategy 3: Fine-tune ST model with LoRA
4. Analyze failure modes (which cell types/timepoints struggle)